In [119]:
from brukerapi.dataset import Dataset as ds
import numpy as np
import matplotlib.pyplot as plt
import glob
from matplotlib import animation
from matplotlib import cm

In [90]:
folders = ["440-459", "461-480"]

rois = np.genfromtxt("ROIset.txt", delimiter=",", dtype=np.int)
f1roi, f2roi, wroi = rois

hahn_paths = sorted(glob.glob("../data/" + folders[0] + "/*/pdata/1/2dseq"))
cpmg_paths = sorted(glob.glob("../data/" + folders[1] + "/*/pdata/1/2dseq"))

method_paths_paths = sorted(glob.glob("../data/" + folders[0] + "/*/method"))
method_paths_paths = sorted(glob.glob("../data/" + folders[1] + "/*/method"))

filter = []

f1hahn_val = []
f2hahn_val = []
whahn_val = []

f1cpmg_val = []
f2cpmg_val = []
wcpmg_val = []

for hahn_path, cpmg_path in zip(hahn_paths, cpmg_paths):

    datahahn = ds(hahn_path)
    datacpmg = ds(cpmg_path)

    datahahn = np.array(datahahn.data)
    datacpmg = np.array(datacpmg.data)

    imhahn = datahahn[:,:,0,1]
    imcpmg = datacpmg[:,:,0,1]

    f1hahn_val.append(np.mean(datahahn[f1roi[1]:f1roi[1]+f1roi[3],f1roi[0]:f1roi[0]+f1roi[2],0,1]))
    f2hahn_val.append(np.mean(datahahn[f2roi[1]:f2roi[1]+f2roi[3],f2roi[0]:f2roi[0]+f2roi[2],0,1]))
    whahn_val.append(np.mean(datahahn[wroi[1]:wroi[1]+wroi[3],wroi[0]:wroi[0]+wroi[2],0,1]))

    f1cpmg_val.append(np.mean(datacpmg[f1roi[1]:f1roi[1]+f1roi[3],f1roi[0]:f1roi[0]+f1roi[2],0,1]))
    f2cpmg_val.append(np.mean(datacpmg[f2roi[1]:f2roi[1]+f2roi[3],f2roi[0]:f2roi[0]+f2roi[2],0,1]))
    wcpmg_val.append(np.mean(datacpmg[wroi[1]:wroi[1]+wroi[3],wroi[0]:wroi[0]+wroi[2],0,1]))

    filter.append(imcpmg-imhahn)

In [44]:
data = ds("../data/461-480/470/pdata/1/2dseq").data

In [116]:
# dejemos solo las fibras, sin el agua, para ver contrastes

f1filtered = []
f2filtered = []

for im in filter:
    aux = np.array(im)
    for i in range(len(im)):
        for j in range(len(im[0])):
            if (j-58)**2 + (i-102)**2 > 28**2:
                aux[i][j]=-1
    f1filtered.append(np.array(aux)[72:132,28:88])

    aux = np.array(im)
    for i in range(len(im)):
        for j in range(len(im[0])):
            if (j-72)**2 + (i-43)**2 > 9**2:
                aux[i][j]=-1
    f2filtered.append(aux)

In [124]:
%matplotlib tk

fig, ax = plt.subplots()

cmap = cm.jet
cmap.set_under("w")

vmax = np.amax(f1filtered)
vmin = 0
cmap = "jet"

ax.imshow(f1filtered[0], cmap=cmap, vmin=0)

def init():
    ax.imshow(f1filtered[0], cmap=cmap, vmin=0)
    ax.set_title("G = 5 $\\frac{mT}{m}")

def animate(i):
    ax.imshow(f1filtered[i], cmap=cmap, vmin=0)
    ax.set_title("G = " + str(i*795/len(filter)) + " $\\frac{mT}{m}$")

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(filter))

Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

anim.save('f1_filter.mp4', writer=writer)

In [78]:
for i,j in filter[0]:
    print(i,j)

ValueError: too many values to unpack (expected 2)